In [1]:
'''
    从es里读取的content是html格式，需要解析成文本
'''
def sentencesMaker(html):
    sentences = []
    if not html or not html.strip():
        return sentences
    try:
        from html.parser import unescape
        html = unescape(html)

        import justext
        paragraphs = justext.justext(html, [])

        cache_sentences = ''

        for p in paragraphs:
            sent = p.text.strip().replace('\xa0', '').replace('\u3000', '')
            sent = sent.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk')
            if not sent:
                continue

            # 可能是含有名字，需要进一步处理
            if len(cache_sentences) < 5:
                cache_sentences += ' ' + sent
            else:
                sentences.append(cache_sentences.strip())
                cache_sentences = sent

        if not not cache_sentences:
            sentences.append(cache_sentences.strip())
    except Exception as e:
        logger.error(e)

    return sentences

In [2]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch('http://ip:port')

In [64]:
from elasticsearch import helpers

# def search_bl2(from_number, off_size):
def search_bl2():
    es_search_options = set_search_optional_bl2()
    es_result = get_search_result(query=es_search_options, index="baidu_bl2")
    return es_result


def get_search_result(query, index):
    es_result = helpers.scan(
        es,
        query=query,
        index=index,
        doc_type='news',
        scroll="10m",
        timeout='10m'
    )
    return es_result


def set_search_optional_bl2(): 
    es_search_options = {
   "query": {
    
    "match_all": {}
  }
  }
    return es_search_options

In [65]:
test_all_results_bl2 = search_bl2()

In [6]:
import jieba
import jieba.posseg as psg

jieba.load_userdict("./user_dict.txt")

import zhon.hanzi
chi_char = zhon.hanzi.punctuation

import string
eng_char = string.punctuation

with open("./stop_words.utf8", "r", encoding="utf-8") as stop_word:
    stop_words = stop_word.readlines()
    
from sklearn.externals import joblib

tfidf = joblib.load("./tfidf_vect_business.pkl")
svm = joblib.load("./svm_business.pkl")

Building prefix dict from /home/lx/anaconda3/lib/python3.6/site-packages/jieba/dict.txt ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.6680512428283691 seconds.
Prefix dict has been built succesfully.


In [66]:
Y = []
i = 0

from functools import reduce

for item in test_all_results_bl2:
    i += 1
    title_content = item["_source"]["title"] + "。" + ''.join(sentencesMaker(item["_source"]["content"]))
    
    token_text = [w.word for w in psg.cut(title_content) if w.flag.startswith('n') 
                  and w.word not in stop_words and w.word not in chi_char and w.word not in eng_char]

    token_text = [reduce(lambda x,y:str(x)+ " " +str(y), token_text)]
    
    test_features = tfidf.transform(token_text)
    
    y_prediction = svm.predict(test_features)
    
    Y.extend(y_prediction.tolist())
    
print(i)

160706


In [67]:
len(Y)

160706

In [68]:
Y[0:10]

['0', '0', '0', '1', '0', '1', '0', '1', '1', '1']

In [69]:
nlabel, plabel = 0, 0
for i in Y:
    if i == "0":
        plabel += 1
    else:
        nlabel += 1
print("百度正样本", plabel, plabel/(plabel + nlabel))
print("百度负样本", nlabel, nlabel/(plabel + nlabel))

百度正样本 48235 0.30014436299827013
百度负样本 112471 0.6998556370017298
